In [25]:
# Implementing freeze-thaw bayesian optimization with two-level Gaussian processes
# A global GP models the asymptotic mean of the learning curves of each HP-config
# Local GPs model the learning curves of each HP-config

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, Kernel, Sum, WhiteKernel
import numpy as np
import matplotlib.pyplot as plt
from ft_testfunction import global_xD, local_xD
import scipy as sc
from freeze_thaw import FreezeThaw, init_configs

# Hyper-Hyperparameters of Freeze-Thaw Bayesian Optimization
ALPHA,BETA = 1,0.5
NOISE = 0.1
B_OLD=10 # 10
B_NEW=3 # 3
N_SAMPLES_MC=1000 # number of samples for Monte Carlo integration
N_FANT=5 # 5 # number of observations we fantasize for each point in the basket
N_INIT_CONFIGS=10 # number of random initializations for the optimization
N_INIT_EPOCHS=5 # number of epochs trained for initial configs
INFERRED_MEAN = 0.8 # inferred mean of the global GP
MATER_NU = 2.5 # Matern kernel parameter
EI_N_SAMPLES = 1000 #10000 # number of samples for EI optimization
PRED_EPOCH = 1 # how many epochs to predict for

# Meta-Parameters of the task
OBS_MIN = 0 # minimal loss value
OBS_MAX = 1 # maximal loss value
bounds={'HP1':(1.,5.),'HP2':(1.,5.)}

In [26]:
# Definition of exponential-decay kernel for local GPs
class ExponentialDecayNoiseKernel(Kernel):
    def __init__(self, alpha=1.0,beta=0.5,noise=0.1):
        self.beta = beta
        self.alpha = alpha
        self.noise = noise
    def __call__(self, X, Y=None,eval_gradient=False):
        if Y is None:
            Y = X
        X=np.array(X)
        Y=np.array(Y)
        return ((self.beta**self.alpha)/((X[None,:]+Y[:,None])+self.beta)**self.alpha + self.noise*np.where(X[None,:]-Y[:,None] == 0, 1, 0)).T
    def diag(self, X):
        return np.diag(self(X))
    def is_stationary(self):
        return False

In [27]:
kernel_global = Matern(nu=MATER_NU)
kernel_local = ExponentialDecayNoiseKernel(alpha=ALPHA,beta=BETA,noise=0.1)

# Start by observing N_INIT_CONFIGS random configurations for N_INIT_EPOCHS epoch each
observed_configs_dicts={}
observed_configs_list=[]
for i in range(N_INIT_CONFIGS):
    new_config=np.empty(0)
    for key in bounds.keys():
        new_config=np.append(new_config,np.round(np.random.uniform(bounds[key][0],bounds[key][1]),2))
    # Observe the new configuration for N_INIT_EPOCHS epochs
    f_space = np.linspace(1,N_INIT_EPOCHS,N_INIT_EPOCHS)
    experimental_data=local_xD(new_config,f_space,noise=0.01)
    observed_configs_dicts['_'.join([str(config) for config in new_config])]=(f_space,experimental_data)
    observed_configs_list.append(new_config)

observed_configs_list=np.array(observed_configs_list)
# print(observed_configs_dicts)
# print(observed_configs_list)

In [31]:
k_n=kernel_local(np.array([0,1,2,3,4]))
np.linalg.inv(k_n)

array([[ 1.39441454, -1.33705004, -0.37995288, -0.09637831,  0.01427635],
       [-1.33705004,  6.84908123, -2.22436332, -1.61549568, -1.22952847],
       [-0.37995288, -2.22436332,  7.93933147, -1.78983552, -1.55495601],
       [-0.09637831, -1.61549568, -1.78983552,  8.27213121, -1.61326105],
       [ 0.01427635, -1.22952847, -1.55495601, -1.61326105,  8.42036313]])

: 